#### Chapter 2: Working with Text
###### Packages that are being used in this notebook:

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

###### This chapter covers data preparation and sampling to get input data "ready" for the LLM

#### 2.1 Understanding word embeddings
###### No code in this section
###### There are many forms of embeddings; we focus on text embeddings in this book
	
###### LLMs work with embeddings in high-dimensional spaces (i.e., thousands of dimensions)
###### Since we can't visualize such high-dimensional spaces (we humans think in 1, 2, or 3 dimensions), the figure below illustrates a 2-dimensional embedding space

#### 2.2 Tokenizing text
###### In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters
	
###### Load raw text we want to work with
###### The Verdict by Edith Wharton is a public domain short story

In [ ]:
import os 
import urllib.request

if not os.path.exists("the-verdict.txt"):
   url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
   file_path = "the-verdict.txt" 
   urllib.request.urlretrieve(url, file_path)

###### (If you encounter an ssl.SSLCertVerificationError when executing the previous code cell, it might be due to using an outdated Python version; you can find more information here on GitHub)

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print(raw_text[:99])

	
###### The goal is to tokenize and embed this text for an LLM
###### Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the text above
###### The following regular expression will split on whitespaces

In [ ]:
import re

text = "Hello, world. This is a test."
result = re.split(r'(\s)', text)

print(result)

###### We don't only want to split on whitespaces but also commas and periods, so let's modify the regular expression to do that as well

In [ ]:
result = re.split(r'([,.]|\s)', text)

print(result)

###### As we can see, this creates empty strings, let's remove them

In [ ]:
# strip whitespace from each item and then filter out any empty strings
result = [item for item in result if item.strip()]
print(result)

###### This looks pretty good, but let's also handle other types of punctuation, such as periods, question marks, and so on

In [ ]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

###### This is pretty good, and we are now ready to apply this tokenization to the raw text

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

###### Let's calculate the total number of tokens

In [ ]:
print(len(preprocessed))

#### 2.3 Converting tokens into token IDs
###### Next, we convert the text tokens into token IDs that we can process via embedding layers later

###### From these tokens, we can now build a vocabulary that consists of all the unique tokens

In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

###### Below are the first 50 entries in this vocabulary:

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i>= 50:
        break

###### Below, we illustrate the tokenization of a short sample text using a small vocabulary:

###### Putting it now all together into a tokenizer class

In [ ]:
class SimpleTokenizerV1:
   def __init__(self, vocab):
       self.str_to_int = vocab
       self.int_to_str = {i:s for s,i in vocab.items()}

   def encode(self, text):
       preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

       preprocessed = [
         item.strip() for item in preprocessed if item.strip()
       ]
       ids = [self.str_to_int[s] for s in preprocessed]
       return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


###### The encode function turns text into token IDs
###### The decode function turns token IDs back into text
###### We can use the tokenizer to encode (that is, tokenize) texts into integers
###### These integers can then be embedded (later) as input of/for the LLM

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

###### We can decode the integers back into text

In [ ]:
tokenizer.decode(ids)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

#### 2.4 Adding special context tokens
###### It's useful to add some "special" tokens for unknown words and to denote the end of a text

###### Some tokenizers use special tokens to help the LLM with additional context

###### Some of these special tokens are

###### [BOS] (beginning of sequence) marks the beginning of text
###### [EOS] (end of sequence) marks where the text ends (this is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles or two different books, and so on)
###### [PAD] (padding) if we train LLMs with a batch size greater than 1 (we may include multiple texts with different lengths; with the padding token we pad the shorter texts to the longest length so that all texts have an equal length)
###### [UNK] to represent words that are not included in the vocabulary

###### Note that GPT-2 does not need any of these tokens mentioned above but only uses an <|endoftext|> token to reduce complexity

###### The <|endoftext|> is analogous to the [EOS] token mentioned above

###### GPT also uses the <|endoftext|> for padding (since we typically use a mask when training on batched inputs, we would not attend padded tokens anyways, so it does not matter what these tokens are)

###### GPT-2 does not use an <UNK> token for out-of-vocabulary words; instead, GPT-2 uses a byte-pair encoding (BPE) tokenizer, which breaks down words into subword units which we will discuss in a later section

###### We use the <|endoftext|> tokens between two independent sources of text:

###### Let's see what happens if we tokenize the following text:

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

###### The above produces an error because the word "Hello" is not contained in the vocabulary
###### To deal with such cases, we can add special tokens like "<|unk|>" to the vocabulary to represent unknown words
###### Since we are already extending the vocabulary, let's add another token called "<|endoftext|>" which is used in GPT-2 training to denote the end of a text (and it's also used between concatenated text, like if our training datasets consists of multiple articles, books, etc.)

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens) }

In [ ]:
len(vocab.items())

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
     print(item)

###### We also need to adjust the tokenizer accordingly so that it knows when and how to use the new <unk> token

In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
       self.str_to_int = vocab
       self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
         preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
         preprocessed = [item.strip() for item in preprocessed if item.strip()]
         preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
         ]

         ids = [self.str_to_int[s] for s in preprocessed]
         return ids

    def decode(self, ids):
        text = " ".join(self.int_to_str[i] for i in ids)
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

###### Let's try to tokenize text with the modified tokenizer:

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = "<|endoftext|>".join((text1, text2))

print(text)

In [ ]:
tokenizer.encode(text)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

#### 2.5 BytePair encoding

###### GPT-2 used BytePair encoding (BPE) as its tokenizer
###### it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
###### For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
###### The original BPE tokenizer can be found here: https://github.com/openai/gpt-2/blob/master/src/encoder.py
###### In this chapter, we are using the BPE tokenizer from OpenAI's open-source tiktoken library, which implements its core algorithms in Rust to improve computational performance
###### I created a notebook in the ./bytepair_encoder that compares these two implementations side-by-side (tiktoken was about 5x faster on the sample text)

In [ ]:
# pip install tiktoken

In [ ]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = (
     "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

In [ ]:
strings = tokenizer.decode(integers)
print(strings)

###### BPE tokenizers break down unknown words into subwords and individual characters:

#### 2.6 Data sampling with a sliding window
###### We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict:

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
   raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

###### For each text chunk, we want the inputs and targets
###### Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right

In [ ]:
enc_sample = enc_text[50:]

In [ ]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size1]

print(f"x: {x}")
print(f"y:      {y}")

###### One by one, the prediction would look like as follows:

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

###### We will take care of the next-word prediction in a later chapter after we covered the attention mechanism
###### For now, we implement a simple data loader that iterates over the input dataset and returns the inputs and targets shifted by one
###### Install and import PyTorch (see Appendix A for installation tips)

In [ ]:
import torch
print("PyTorch version:", torch.__version__)

###### We use a sliding window approach, changing the position by +1:

###### Create dataset and dataloader that extract chunks from the input text dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequence of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i+ max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
       return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
     # Initialize the tokenizer
     tokenizer = tiktoken.get_encoding("gpt2")

     # create Dataset
     dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

     # Create DataLoader
     dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
     )

     return dataloader

###### Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4: